In [0]:
GRID_PATH = '/datascope/subaru/data/pfsspec/models/stellar/grid/phoenix/phoenix_HiRes'

In [0]:
import os, sys

# Allow load project as module
sys.path.insert(0, '../../../../python')

In [0]:
%matplotlib inline

In [0]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import h5py

# Open stellar grid

In [0]:
from pfs.ga.pfsspec.core.grid import ArrayGrid
from pfs.ga.pfsspec.stellar.grid import ModelGrid
from pfs.ga.pfsspec.stellar.grid.bosz import Bosz
from pfs.ga.pfsspec.stellar.grid.phoenix import Phoenix

In [0]:
fn = os.path.join(GRID_PATH, 'spectra.h5')
grid = ModelGrid(Phoenix(), ArrayGrid)
grid.wave_lim = [3000, 9000]
grid.preload_arrays = False
grid.load(fn, format='h5')

# Resampling

In [0]:
from pfs.ga.pfsspec.core.obsmod.psf import VelocityDispersion
from pfs.ga.pfsspec.core.obsmod.resampling import *

In [0]:
model = grid.get_model(M_H=0, T_eff=4500, log_g=1, a_M=0, )
model.wave.shape, model.flux.shape, model.wave[0], model.wave[-1]

psf = VelocityDispersion(vdisp=1, reuse_kernel=True)
s = psf.get_optimal_size(model.wave)

model.convolve_psf(psf, size=s)

In [0]:
wave_edges = np.linspace(8000, 8500, 501)
wave = 0.5 * (wave_edges[1:] + wave_edges[:-1])

res = Interp1dResampler()
model_interp1d = model.copy()
model_interp1d.apply_resampler(res, wave, wave_edges)

res = FluxConservingResampler()
model_cumsum = model.copy()
model_cumsum.apply_resampler(res, wave, wave_edges)

res = PysynphotResampler()
model_synphot = model.copy()
model_synphot.apply_resampler(res, wave, wave_edges)

In [0]:
f, ax = plt.subplots(1, 1, figsize=(5, 2.5), dpi=300)

ax.step(model.wave, model.flux, where='mid', lw=0.3, label='model')
ax.step(model_interp1d.wave, model_interp1d.flux, where='mid', lw=0.3, label='interp1d')
ax.step(model_cumsum.wave, model_cumsum.flux, where='mid', lw=0.3, label='cumsum trick')
ax.step(model_synphot.wave, model_synphot.flux, where='mid', lw=0.3, label='synphot')

ax.set_xlim(8200, 8215)
ax.set_ylim(0.8e14, 2.0e14)